In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [ ]:
data = pd.read_csv("./gurgaon_properties_outlier_treated.csv")
data.head()

In [ ]:
data.isnull().sum()

# Built up area

In [ ]:
sns.scatterplot(data=data, x="built_up_area", y="super_built_up_area")
plt.show()

In [ ]:
sns.scatterplot(data=data, x="built_up_area", y="carpet_area")
plt.show()

In [ ]:
data[(data["super_built_up_area"].isnull()) & (data["built_up_area"].isnull()) & (data["carpet_area"].isnull())]

In [ ]:
# filtering rows where all the three areas are non-null

all_present_df = data[~((data["super_built_up_area"].isnull()) | (data["built_up_area"].isnull()) | (data["carpet_area"].isnull()))]

In [ ]:
super_to_builtup_ratio = (all_present_df["super_built_up_area"] / all_present_df["built_up_area"]).median()

In [ ]:
carpet_to_builtup_ratio = (all_present_df["carpet_area"] / all_present_df["built_up_area"]).median()

In [ ]:
# builtup area null , super builtup and carpet area present
sbc_df = data[~(data["super_built_up_area"].isnull()) & ~(data["carpet_area"].isnull()) & (data["built_up_area"].isnull())]

In [ ]:
sbc_df["built_up_area"].fillna(round(((sbc_df["super_built_up_area"] / super_to_builtup_ratio) + (sbc_df["carpet_area"] / carpet_to_builtup_ratio)) / 2), inplace=True)

In [ ]:
data.update(sbc_df)

In [ ]:
data["built_up_area"].isna().sum()

In [ ]:
#filtering data where super builtup area is available, carpet and builtup areas are null
sb_df = data[~(data["super_built_up_area"].isnull()) & (data["built_up_area"].isnull()) & (data["carpet_area"].isnull())]

In [ ]:
sb_df["built_up_area"].fillna(round(sb_df["super_built_up_area"] / super_to_builtup_ratio), inplace=True)

In [ ]:
data.update(sb_df)

In [ ]:
data["built_up_area"].isna().sum()

In [ ]:
# only carpet area present, rest are null

c_df= data[(data["super_built_up_area"].isnull()) & (data["built_up_area"].isnull()) & ~(data["carpet_area"].isnull())]

In [ ]:
c_df["built_up_area"].fillna(round(c_df["carpet_area"] / carpet_to_builtup_ratio), inplace=True)

In [ ]:
data.update(c_df)

In [ ]:
data["built_up_area"].isna().sum()

# Analyzing builtup area and price

In [ ]:
sns.scatterplot(data=data, x="built_up_area", y="price")
plt.show()

- For the most part, as builtup area increases, price increases -> seems logical
- There are some properties where builtup area < 2000 but price goes up till 25 crores.

In [ ]:
anomaly_df = data[(data["built_up_area"] < 2000) & (data["price"] > 2.5)][["price", "area", "built_up_area"]]

- Upon analyzing these anomalies, it seemed that area values justified the price, but builtup area values didnt.
- So, I simply replaced the builtup area values with area values.

In [ ]:
anomaly_df["built_up_area"] = anomaly_df["area"]

In [ ]:
data.update(anomaly_df)

In [ ]:
sns.scatterplot(data=data, x="built_up_area", y="price")
plt.title("After treating anomalies")
plt.show()

In [ ]:
data.columns

In [ ]:
# dropping some unnecessary columns
data.drop(columns=["area", "areaWithType" , "super_built_up_area", "carpet_area", "area_room_ratio"], inplace=True)

In [ ]:
data.shape

# floorNum

In [ ]:
data["floorNum"].isnull().sum()

In [ ]:
data[data["floorNum"].isnull()]

- there are 17 rows where floorNum value is missing
- 15 of these are houses
- filling these null values with median number of floors of all houses in the data

In [ ]:
data[data["property_type"] == "house"]["floorNum"].median()

In [ ]:
data["floorNum"].fillna(2, inplace=True)

In [ ]:
data["floorNum"].isnull().sum()

# facing

In [ ]:
data["facing"].isnull().sum()

In [ ]:
data["facing"].value_counts().plot(kind="pie", autopct="%.2f%%")
plt.show()

In [ ]:
data["facing"].isnull().sum() / len(data)

- 28% values are missing
- facing is not a crucial feature for predicting price
- dropping facing column

In [ ]:
data.drop(columns="facing", inplace=True)

# society

In [ ]:
data["society"].isnull().sum()

In [ ]:
data[data["society"].isnull()]

In [ ]:
data.drop(index=2536, inplace=True)

In [ ]:
data["society"].isnull().sum()

# agePossession

In [ ]:
data["agePossession"].value_counts()

In [ ]:
def mode_based_imputation(row):
    """fills missing values of agePossession column based on sector and property type columns"""
    if row["agePossession"] == "Undefined":
        mode = data[(data["sector"] == row["sector"]) & (data["property_type"] == row["property_type"])]["agePossession"].mode()
        if not mode.empty:
            return mode.iat[0]
        else:
            return np.nan
    else:
        return row["agePossession"]

In [ ]:
data["agePossession"] = data.apply(mode_based_imputation, axis=1)

In [ ]:
data["agePossession"].value_counts()

In [ ]:
data[data["agePossession"] == "Undefined"]

In [ ]:
def mode_based_imputation2(row):
    """fills null values of agePossession column based on sector only"""
    if row["agePossession"] == "Undefined":
        mode = data[data["sector"] == row["sector"]]["agePossession"].mode()
        if not mode.empty:
            return mode.iat[0]
        else:
            return np.nan
    else:
        return row["agePossession"]

In [ ]:
data["agePossession"] = data.apply(mode_based_imputation2, axis=1)

In [ ]:
data["agePossession"].value_counts()

In [ ]:
def mode_based_imputation3(row):
    """fills null values of agePossession column based on property type only"""
    if row["agePossession"] == "Undefined":
        mode = data[data["property_type"] == row["property_type"]]["agePossession"].mode()
        if not mode.empty:
            return mode.iat[0]
        else:
            return np.nan
    else:
        return row["agePossession"]

In [ ]:
data["agePossession"] = data.apply(mode_based_imputation3, axis=1)

In [ ]:
data["agePossession"].value_counts()

In [ ]:
# exporting as csv file
data.to_csv("gurgaon_properties_after_missing_value_imputation.csv")